In [ ]:
import numpy as np
import pandas as pd
from autumn.infrastructure.remote import springboard
from aust_covid.calibration import get_outcome_df_by_chain
import seaborn as sns
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from inputs.constants import RUN_IDS, RUNS_PATH
from emutools.plotting import get_row_col_for_subplots
from emutools.calibration import get_like_components, plot_like_components_by_analysis
pd.options.plotting.backend = 'plotly'

In [ ]:
def plot_indicator_progression(like_dfs, measure):
    n_cols = 2
    fig = make_subplots(rows=2, cols=n_cols, subplot_titles=list(RUN_IDS.keys()), shared_yaxes=True)
    for i, analysis in enumerate(RUN_IDS.keys()):
        row, col = get_row_col_for_subplots(i, n_cols)
        fig.add_traces(like_dfs[analysis][measure].plot().data, rows=row, cols=col)
    fig.update_layout(height=1000, title={'text': measure})
    return fig

In [ ]:
components = ['logposterior', 'logprior', 'loglikelihood', 'll_adult_seropos_prop', 'll_deaths_ma', 'll_notifications_ma']
burn_in = 5000
like_outputs = get_like_components(components)
clips = {
    'logposterior': -70.0, 
    'logprior': -46.0, 
    'loglikelihood': -28.0,
    'll_adult_seropos_prop': -4.0, 
    'll_deaths_ma': -10.0, 
    'll_notifications_ma': -17.0,
}
plot_like_components_by_analysis(like_outputs, 'kdeplot', burn_in, clips=clips);

In [ ]:
plot_like_components_by_analysis(like_outputs, components, 'violinplot', burn_in);

In [ ]:
plot_like_components_by_analysis(like_outputs, components, 'histplot', burn_in);

In [ ]:
like_dfs = get_outcome_df_by_chain()
plot_indicator_progression(like_dfs, 'logposterior')